La base de datos House_price.csv contiene información de casas. La idea es obtener un modelo que permita predecir el valor de venta de una casa. Las variables son las siguientes:
•          LotFrontage: Pies lineales de calle conectados a la propiedad
•          LotArea: Tamaño del lote en pies cuadrados
•          MasVnrArea: Área de revestimiento de mampostería en pies cuadrados
•          BsmtFinSF1: Pies cuadrados del sótano terminados tipo 1
•          BsmtFinSF2: Pies cuadrados con del sótano acabados tipo 2
•          BsmtUnfSF: Pies cuadrados sin terminar de área del sótano
•          TotalBsmtSF: Total de pies cuadrados de área del sótano
•          1stFlrSF: Pies cuadrados del primer piso
•          2ndFlrSF: Pies cuadrados del segundo piso
•          LowQualFinSF: Pies cuadrados con acabado de baja calidad (todos los pisos)
•          GrLivArea: Pies cuadrados de área habitable sobre el nivel (suelo)
•          GarageArea: Tamaño del garaje en pies cuadrados
•          WoodDeckSF: Área de la plataforma de madera en pies cuadradosActividad 4
•          OpenPorchSF: Área del porche abierto en pies cuadrados
•          EnclosedPorch: Área del porche cerrado en pies cuadrados
•          3SsnPorch: área de porche de tres estaciones en pies cuadrados
•          ScreenPorch: Área del porche de la pantalla en pies cuadrados
•          PoolArea: Área de la piscina en pies cuadrados
•          MiscVal: Valor de la función miscelánea
•          SalePrice: el precio de venta de la propiedad en dólares. Esta es la variable objetivo que se quiere modelar.

a) Cargue la base de datos, ¿hay alguna columna que no sea útil para el análisis? Revise si existen casos faltantes en la base de datos, según la descripción  de  las  variables,  ¿qué  pudiera  significar  un  NA?  Tome decisiones de qué hacer si existen casos faltantes. Si desea imputar los valores  nulos  de  alguna  variable  por  un  valor  puede  utilizar data["columna"].fillna('valor', inplace = True).

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from datetime import datetime

#ruta del data
house_price = "./datos/House_price.csv"

# Cargar la base de datos
data = pd.read_csv(house_price, delimiter=';')

# Imprimir las primeras filas para ver qué columnas hay
print(data.head())

# Verificar si existen casos faltantes
print(data.isnull().sum())

# Eliminar la columna "Id" (no útil para el análisis)
data.drop(columns=["Id"], inplace=True)

# Tratar valores faltantes (NA)
# Imputar los valores nulos en todas las columnas con un valor específico (por ejemplo, "valor")
data = data.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x.fillna('valor'))




   Id  LotFrontage  LotArea  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  \
0   1         65.0     8450       196.0         706           0        150   
1   2         80.0     9600         0.0         978           0        284   
2   3         68.0    11250       162.0         486           0        434   
3   4         60.0     9550         0.0         216           0        540   
4   5         84.0    14260       350.0         655           0        490   

   TotalBsmtSF  1stFlrSF  2ndFlrSF  ...  GrLivArea  GarageArea  WoodDeckSF  \
0          856       856       854  ...       1710         548           0   
1         1262      1262         0  ...       1262         460         298   
2          920       920       866  ...       1786         608           0   
3          756       961       756  ...       1717         642           0   
4         1145      1145      1053  ...       2198         836         192   

   OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolAre

b) La variable a modelar corresponde al precio de venta de la casa en dólares. En base a los modelos que hemos aprendido en clases, ¿qué modelo(s) podría(n) ser de utilidad para predecir el precio de venta de casas mediante las variables predictoras?

b) Modelos para predecir el precio de venta de casas:

Dado que queremos predecir el precio de venta de casas, los modelos de regresión serían apropiados. Algunos modelos útiles podrían ser:
Regresión lineal
Regresión Ridge
Regresión Lasso
Regresión de árboles (como Random Forest)

c) Vamos a implementar un modelo de regresión lineal para modelar el precio de venta de casas. Considerando que en la base de datos tenemos varias variables que dependen de otras, ¿cuál pudiera ser el problema que surgiría en este contexto? ¿por qué esto es un problema? Discuta.

c) Problema de dependencia entre variables:

La multicolinealidad entre las variables predictoras es un problema en el análisis de regresión. Ocurre cuando dos o más variables independientes están altamente correlacionadas entre sí. Algunos puntos clave sobre este problema son:

Interpretación de Coeficientes:
En un modelo de regresión lineal, los coeficientes representan el cambio en la variable dependiente (precio de venta en este caso) por cada unidad de cambio en la variable independiente.
Si dos variables predictoras están altamente correlacionadas, se vuelve difícil determinar cuál de ellas está realmente influyendo en la variable objetivo.
Inestabilidad de Coeficientes:
La multicolinealidad puede hacer que los coeficientes sean inestables y sensibles a pequeñas variaciones en los datos.
Esto dificulta la interpretación y la confiabilidad del modelo.

d) Realice un análisis de la correlación entre las variables. Comente, ¿existen variables muy correlacionadas entre sí? ¿Por qué cree que ocurre?

In [2]:
# Calcular la matriz de correlación
correlation_matrix = data.corr()

# Filtrar las correlaciones fuertes (mayores que 0.7 o menores que -0.7)
strong_correlations = correlation_matrix[(correlation_matrix > 0.7) | (correlation_matrix < -0.7)]

# Imprimir las correlaciones fuertes
print(strong_correlations)

               LotFrontage  LotArea  MasVnrArea  BsmtFinSF1  BsmtFinSF2  \
LotFrontage            1.0      NaN         NaN         NaN         NaN   
LotArea                NaN      1.0         NaN         NaN         NaN   
MasVnrArea             NaN      NaN         1.0         NaN         NaN   
BsmtFinSF1             NaN      NaN         NaN         1.0         NaN   
BsmtFinSF2             NaN      NaN         NaN         NaN         1.0   
BsmtUnfSF              NaN      NaN         NaN         NaN         NaN   
TotalBsmtSF            NaN      NaN         NaN         NaN         NaN   
1stFlrSF               NaN      NaN         NaN         NaN         NaN   
2ndFlrSF               NaN      NaN         NaN         NaN         NaN   
LowQualFinSF           NaN      NaN         NaN         NaN         NaN   
GrLivArea              NaN      NaN         NaN         NaN         NaN   
GarageArea             NaN      NaN         NaN         NaN         NaN   
WoodDeckSF             Na

TotalBsmtSF y 1stFlrSF:
Existe una alta correlación positiva (0.82) entre el área total del sótano (TotalBsmtSF) y el área del primer piso (1stFlrSF). Esto es comprensible, ya que el área total del sótano generalmente incluye el área del primer piso.
GrLivArea y 2ndFlrSF:
Hay una correlación positiva (0.69) entre el área habitable sobre el nivel del suelo (GrLivArea) y el área del segundo piso (2ndFlrSF). Esto también tiene sentido, ya que el área habitable total a menudo incluye el segundo piso.
GrLivArea y SalePrice:
Existe una correlación positiva fuerte (0.71) entre el área habitable sobre el nivel del suelo (GrLivArea) y el precio de venta (SalePrice). Esto sugiere que, en general, las casas más grandes tienden a tener un precio de venta más alto.

In [3]:

# Separar variables predictoras (X) y variable objetivo (y)
X = data.drop(columns=["SalePrice"])
y = data["SalePrice"]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarizar las variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Obtener el tiempo de procesamiento para la regresión lineal
start = datetime.now()
regression_model = LinearRegression()
regression_model.fit(X_train_scaled, y_train)
print("Tiempo de procesamiento para la regresión lineal:", datetime.now() - start)



Tiempo de procesamiento para la regresión lineal: 0:00:00.004882


f) Obtenga  las  componentes  principales  utilizando  las  variables estandarizadas, si desea explicar un 80% de la variabilidad, ¿cuántas componentes debería elegir? Argumente.

In [4]:

# Obtener las componentes principales
pca = PCA(n_components=0.8)  # Explicar al menos el 80% de la variabilidad
X_train_pca = pca.fit_transform(X_train_scaled)


g) Corra el modelo de regresión lineal con las componentes principales como variables predictoras. Vuelva a calcular el tiempo de procesamiento. ¿En cuál  caso  suele  demorarse  menos?  ¿Qué  ganamos  al  realizar componentes principales? Concluya

In [5]:
# Correr el modelo de regresión lineal con las componentes principales
start_pca = datetime.now()
regression_model_pca = LinearRegression()
regression_model_pca.fit(X_train_pca, y_train)
print("Tiempo de procesamiento con componentes principales:", datetime.now() - start_pca)

Tiempo de procesamiento con componentes principales: 0:00:00.002930


Reducción de dimensionalidad:
El PCA nos permite reducir la cantidad de variables originales (dimensiones) en nuestros datos.
Al seleccionar un número menor de componentes principales, simplificamos el modelo y mejoramos la eficiencia computacional.
Eliminación de multicolinealidad:
Las componentes principales son ortogonales entre sí, lo que ayuda a reducir la multicolinealidad entre las variables originales.
Esto mejora la interpretación de los coeficientes en los modelos de regresión.
Conservación de la variabilidad:
A pesar de reducir la dimensionalidad, el PCA intenta retener la mayor cantidad posible de la variabilidad original en los datos.
Al elegir un número suficiente de componentes principales, podemos explicar una gran parte de la variabilidad total.
En conclusión, al realizar componentes principales, ganamos una forma más eficiente de representar nuestros datos, reduciendo la complejidad y manteniendo la información relevante. Sin embargo, debemos equilibrar esto con la cantidad de variabilidad que estamos dispuestos a perder.